In [2]:
import pandas as pd

# Hard-coded file name
csv_path = "mydata.csv"   # <-- change this to your file name

# Number of rows to show
num_rows = 5

try:
    df = pd.read_csv(csv_path)
    print(df.head(num_rows))
except Exception as e:
    print("Error:", e)


/tmp/ipython-input-3420948779.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


   Unnamed: 0   timestamp device_ip_address remote_ip_address  \
0           0  1760574340         10.7.0.24      10.83.12.118   
1           1  1760574340         10.7.0.24       17.57.147.6   
2           2  1760574350         10.7.0.24      10.83.12.118   
3           3  1760574350         10.7.0.24     17.253.97.208   
4           4  1760574350         10.7.0.24       17.57.147.6   

          remote_hostname remote_port protocol  upload_byte_count  \
0                 pi.hole          53      udp               1051   
1  courier.push.apple.com        5223      tcp               1396   
2                 pi.hole          53      udp                198   
3    gsp-ssl.ls.apple.com         443      tcp               3355   
4  courier.push.apple.com        5223      tcp              32241   

   upload_packet_count  download_byte_count  download_packet_count  \
0                   16                 1638                     12   
1                    3                 3856           

In [3]:
import pandas as pd
from datetime import datetime, timedelta

# ---- CONFIG ----
CSV_FILE = "mydata.csv"     # <-- replace with your CSV filename
UPLOAD_THRESHOLD = 1_000_000  # bytes (1000 KB)
TIME_STEP = 1                 # minutes per log entry (adjust if needed)

# ----------------

df = pd.read_csv(CSV_FILE)

# Convert timestamp (epoch) → datetime if needed
if df['timestamp'].dtype != 'datetime64[ns]':
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Mark screen ON where upload > threshold
df["screen_on"] = df["upload_byte_count"] > UPLOAD_THRESHOLD

# Sort by time
df = df.sort_values("timestamp")

# Estimate screen-on duration per entry
df["screen_on_minutes"] = df["screen_on"] * TIME_STEP

# Group by date
df["date"] = df["timestamp"].dt.date
daily_screen_time = df.groupby("date")["screen_on_minutes"].sum()

print("\n=== Estimated Daily Screen Time (minutes) ===\n")
print(daily_screen_time)
print("\n=============================================\n")


/tmp/ipython-input-3156564230.py:11: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE)



=== Estimated Daily Screen Time (minutes) ===

date
2025-10-16     41
2025-10-17    725
2025-10-18     12
2025-10-19    164
2025-10-20    148
2025-10-21    202
2025-10-22    120
2025-10-23    302
2025-10-24     53
2025-10-25    107
2025-10-26     66
2025-10-27    226
2025-10-28      0
2025-10-29     29
2025-10-30    242
2025-10-31    136
2025-11-01      6
2025-11-02     89
2025-11-03      9
2025-11-04    223
2025-11-05    101
2025-11-06    103
2025-11-07    117
2025-11-08     22
2025-11-09    474
2025-11-10    202
2025-11-11     33
2025-11-12    128
2025-11-13    273
2025-11-14    501
2025-11-15    145
2025-11-16      9
2025-11-17    147
2025-11-18    144
2025-11-19     16
2025-11-20     15
2025-11-21      4
2025-11-22    476
2025-11-23    303
2025-11-24      9
2025-11-25      8
2025-11-26     40
2025-11-27      2
2025-11-28     18
2025-11-29     85
2025-11-30     57
2025-12-01    124
2025-12-02     30
2025-12-03     98
Name: screen_on_minutes, dtype: int64




In [4]:
import pandas as pd
from datetime import timedelta

# ---- CONFIG ----
CSV_FILE = "mydata.csv"              # <-- change to your file
OUTPUT_FILE = "daily_screen_time.csv"  # output CSV
UPLOAD_THRESHOLD = 1_000_000           # 1000 KB = 1,000,000 bytes
TIME_STEP_MIN = 1                      # minutes per log entry (adjust if needed)
# ----------------

# Load CSV
df = pd.read_csv(CSV_FILE)

# Convert timestamp epoch → datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Mark screen ON when upload threshold crossed
df["screen_on"] = df["upload_byte_count"] > UPLOAD_THRESHOLD

# Sort chronologically
df = df.sort_values("timestamp")

# Add minutes for each screen-on entry
df["screen_on_minutes"] = df["screen_on"] * TIME_STEP_MIN

# Extract date for grouping
df["date"] = df["timestamp"].dt.date

# Group & compute total minutes per day
daily_minutes = df.groupby("date")["screen_on_minutes"].sum().reset_index()

# Convert minutes → Hours + Minutes
daily_minutes["hours"] = (daily_minutes["screen_on_minutes"] // 60).astype(int)
daily_minutes["minutes"] = (daily_minutes["screen_on_minutes"] % 60).astype(int)

# Format the screen time as "Xh Ym"
daily_minutes["screen_time_formatted"] = daily_minutes["hours"].astype(str) + "h " + daily_minutes["minutes"].astype(str) + "m"

# Save to CSV
daily_minutes.to_csv(OUTPUT_FILE, index=False)

print("\n✅ Screen-time CSV created:", OUTPUT_FILE)
print("\n=== Daily Screen Time ===\n")
print(daily_minutes)


/tmp/ipython-input-4125293054.py:12: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE)



✅ Screen-time CSV created: daily_screen_time.csv

=== Daily Screen Time ===

          date  screen_on_minutes  hours  minutes screen_time_formatted
0   2025-10-16                 41      0       41                0h 41m
1   2025-10-17                725     12        5                12h 5m
2   2025-10-18                 12      0       12                0h 12m
3   2025-10-19                164      2       44                2h 44m
4   2025-10-20                148      2       28                2h 28m
5   2025-10-21                202      3       22                3h 22m
6   2025-10-22                120      2        0                 2h 0m
7   2025-10-23                302      5        2                 5h 2m
8   2025-10-24                 53      0       53                0h 53m
9   2025-10-25                107      1       47                1h 47m
10  2025-10-26                 66      1        6                 1h 6m
11  2025-10-27                226      3       46         

In [9]:
import pandas as pd

# -------------- CONFIG --------------
CSV_FILE = "mydata.csv"                   # <- input CSV
OUTPUT_MINUTE_FILE = "minute_screen_time.csv"
OUTPUT_DAILY_FILE = "daily_screen_time.csv"
UPLOAD_THRESHOLD_PER_MIN = 1_000_000        # 1000 KB per minute
# ------------------------------------


# Load data
df = pd.read_csv(CSV_FILE)

# Convert timestamp (epoch seconds) to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

# Set index for resampling
df = df.set_index("timestamp").sort_index()

# 1) Minute-by-minute upload sum
#    This creates a row for every minute between first and last timestamp.
minute = df.resample("1T").agg({"upload_byte_count": "sum"})

# 2) Apply your rule PER MINUTE
minute["screen_on"] = (minute["upload_byte_count"] > UPLOAD_THRESHOLD_PER_MIN).astype(int)

# 3) Save minute-by-minute data
minute.to_csv(OUTPUT_MINUTE_FILE)
print(f"✅ Minute-by-minute screen data saved to: {OUTPUT_MINUTE_FILE}")

# 4) Daily totals from minute data
minute["date"] = minute.index.date
daily = minute.groupby("date")["screen_on"].sum().reset_index()
daily.rename(columns={"screen_on": "screen_on_minutes"}, inplace=True)

# 5) Convert minutes -> hours + minutes
daily["hours"] = (daily["screen_on_minutes"] // 60).astype(int)
daily["minutes"] = (daily["screen_on_minutes"] % 60).astype(int)
daily["screen_time_formatted"] = (
    daily["hours"].astype(str) + "h " + daily["minutes"].astype(str) + "m"
)

# 6) Save daily summary
daily.to_csv(OUTPUT_DAILY_FILE, index=False)
print(f"✅ Daily screen-time summary saved to: {OUTPUT_DAILY_FILE}")

print("\n=== Daily Screen Time (approx) ===")
print(daily)


/tmp/ipython-input-2944330259.py:12: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE)
/tmp/ipython-input-2944330259.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  minute = df.resample("1T").agg({"upload_byte_count": "sum"})


✅ Minute-by-minute screen data saved to: minute_screen_time.csv
✅ Daily screen-time summary saved to: daily_screen_time.csv

=== Daily Screen Time (approx) ===
          date  screen_on_minutes  hours  minutes screen_time_formatted
0   2025-10-16                 83      1       23                1h 23m
1   2025-10-17                129      2        9                 2h 9m
2   2025-10-18                 52      0       52                0h 52m
3   2025-10-19                 69      1        9                 1h 9m
4   2025-10-20                 97      1       37                1h 37m
5   2025-10-21                 96      1       36                1h 36m
6   2025-10-22                 84      1       24                1h 24m
7   2025-10-23                142      2       22                2h 22m
8   2025-10-24                 55      0       55                0h 55m
9   2025-10-25                 87      1       27                1h 27m
10  2025-10-26                 53      0       5